In [64]:
import pandas as pd, numpy as np, sqlite3
data = pd.read_csv("./data/taxes.csv")


In [15]:
data.columns

Index(['TIPO DE PRODUCTO:', 'Gravámenes Vigentes', 'Ad / Valorem',
       'Impuesto Selectivo al Consumo', 'Impuesto General a las Ventas',
       'Impuesto de Promoción Municipal', 'Derecho Específicos',
       'Derecho Antidumping', 'Seguro', 'Sobretasa Tributo',
       'Unidad de Medida:', 'Sobretasa Sanción', 'hs_code'],
      dtype='object')

In [17]:

data.head(2)

,TIPO DE PRODUCTO:,Gravámenes Vigentes,Ad / Valorem,Impuesto Selectivo al Consumo,Impuesto General a las Ventas,Impuesto de Promoción Municipal,Derecho Específicos,Derecho Antidumping,Seguro,Sobretasa Tributo,Unidad de Medida:,Sobretasa Sanción,hs_code
0,LEY 29666-IGV 20.02.11,Valor,0%,0%,16%,2%,0,0,2%,0%,(*),0.0,2515110000
1,LEY 29666-IGV 20.02.11,Valor,0%,0%,16%,2%,0,0,2%,0%,(*),0.0,2515200000


In [65]:
cl = [
    "ley",
    "gv",
    "ad_valorem", 'excise_tax', 'igv', 'ipm', 'sp_tax', 'adp_tax', 'insurage', 'surcharge_tax', "und", "penalty_charge", "hs_code"
]

data = pd.read_csv("./data/taxes.csv")
percent_columns = ["ad_valorem", 'excise_tax', 'igv', 'ipm', 'sp_tax', 'adp_tax', 'insurage', 'surcharge_tax', 'penalty_charge']
data.columns = cl
# data[percent_columns] = 
data[percent_columns] = data[percent_columns].replace('N.A.', 0)

data[percent_columns] = data[percent_columns].replace('Detalle', None)
data[percent_columns] = data[percent_columns].replace('S', None)
data[percent_columns] = data[percent_columns].replace('%', "", regex=True).astype(float) / 100

data = data.drop(columns=['gv', 'und'])

C:\Users\Jhon\AppData\Local\Temp\ipykernel_2428\3909525534.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[percent_columns] = data[percent_columns].replace('N.A.', 0)
C:\Users\Jhon\AppData\Local\Temp\ipykernel_2428\3909525534.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[percent_columns] = data[percent_columns].replace('%', "", regex=True).astype(float) / 100


In [62]:
sql_data = "./base.sqlite"
conn = sqlite3.connect(sql_data)

data.to_sql("taxes", conn, index=False, if_exists='replace')
conn.close()


In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import pandas as pd

In [22]:
url = 'http://www.aduanet.gob.pe/itarancel/arancelS01Alias'
browser = webdriver.Chrome()
browser.get(url)

In [23]:
hs = '2935500000'
hs_input = "/html/body/form/div/table/tbody/tr[1]/td/input[1]"
consult_click = "/html/body/form/div/table/tbody/tr[3]/td/input[1]"

browser.find_element(By.XPATH, hs_input).send_keys(hs)
browser.find_element(By.XPATH, consult_click).click()

In [30]:
browser.find_element(By.XPATH, hs_input).clear()

In [6]:
# Rescatar los valores

description = browser.find_element(By.ID, "descripcionFrame")
browser.switch_to.frame(description)
table = browser.find_element(By.ID, "descripcionFrame22")

browser.switch_to.frame(table)

In [8]:
tbody = browser.find_element(By.XPATH, "/html/body/div/center/table/tbody")
data = []

for tr in tbody.find_elements(By.XPATH, "//tr"):
    row = [x.text for x in tr.find_elements(By.XPATH, ".//td")]
    data.append(row)


In [37]:
all = pd.DataFrame(data)
# all['hs'] = hs

In [43]:
allt = all.transpose().reset_index(drop=True)
allt.columns = allt.iloc[0]
allt = allt[1:]
allt['hs_code'] = hs

In [44]:
allt

,TIPO DE PRODUCTO:,Gravámenes Vigentes,Ad / Valorem,Impuesto Selectivo al Consumo,Impuesto General a las Ventas,Impuesto de Promoción Municipal,Derecho Específicos,Derecho Antidumping,Seguro,Sobretasa Tributo,Unidad de Medida:,Sobretasa Sanción,hs_code
1,DS.342-2016-EF-LEY 29666-IGV 20.02.11,Valor,0%,0%,16%,2%,N.A.,N.A.,1.25%,0%,(*),N.A.,2934101000


In [16]:
browser.switch_to.default_content()
browser.switch_to.default_content()


In [31]:
hs = '2934101000'
hs_input = "/html/body/form/div/table/tbody/tr[1]/td/input[1]"
consult_click = "/html/body/form/div/table/tbody/tr[3]/td/input[1]"

browser.find_element(By.XPATH, hs_input).send_keys(hs)
browser.find_element(By.XPATH, consult_click).click()